Before running the file Upload all your data set on your goole drive in a zip format

In [9]:
import glob
import os
import cv2
import face_recognition
from tqdm.autonotebook import tqdm

# Extract frames from video
def frame_extract(path):
    vidObj = cv2.VideoCapture(path)
    success, image = vidObj.read()
    while success:
        yield image
        success, image = vidObj.read()

# Process videos and create face videos
def create_face_videos(path_list, out_dir):
    os.makedirs(out_dir, exist_ok=True)  # Ensure output directory exists
    already_present_count = glob.glob(out_dir + '*.mp4') #directory of videos to be changed
    print("No of videos already present:", len(already_present_count))

    for path in tqdm(path_list):
        out_path = os.path.join(out_dir, os.path.basename(path))
        if os.path.exists(out_path):
            print("File Already exists:", out_path)
            continue

        out = cv2.VideoWriter(out_path, cv2.VideoWriter_fourcc(*'MJPG'), 30, (112, 112))
        frames = []

        for idx, frame in enumerate(frame_extract(path)):
            if idx <= 150:  # Use the first 150 frames
                frames.append(frame)
                if len(frames) == 4:
                    faces = face_recognition.batch_face_locations(frames)
                    for i, face in enumerate(faces):
                        if len(face) != 0:
                            top, right, bottom, left = face[0]
                            try:
                                cropped_face = cv2.resize(frames[i][top:bottom, left:right, :], (112, 112))
                                out.write(cropped_face)
                            except Exception as e:
                                print(f"Error processing frame: {e}")
                    frames = []
        out.release()

# Get video files in current directory
video_files = glob.glob('./*.mp4')
print("Video files:", video_files)

# Output directory for processed videos
output_dir = './processed_videos/'

# Process the videos
create_face_videos(video_files, output_dir)


Video files: ['./WhatsApp Video 2024-11-25 at 19.18.43_b43b1e34.mp4']
No of videos already present: 0


  0%|          | 0/1 [00:00<?, ?it/s]